In [1]:
!pip install anytree

In [2]:
import pandas as pd
from anytree import Node, RenderTree, AsciiStyle, PreOrderIter
from copy import deepcopy

In [3]:
# Definir el número máximo de niveles
X = 2

# Leer el archivo CSV
df = pd.read_csv('FinalDataset/DBpediaClasses.csv')

In [4]:
# Crear un diccionario para almacenar los nodos
nodes = {}

# Construir los nodos y el árbol
for _, row in df.iterrows():
    parent_name = row['SubClass']
    child_name = row['Class']
    
    # Crear el nodo padre si no existe
    if parent_name not in nodes:
        nodes[parent_name] = Node(parent_name)
    
    # Crear el nodo hijo y asignarlo al nodo padre
    if child_name not in nodes:
        nodes[child_name] = Node(child_name, parent=nodes[parent_name])
    else:
        nodes[child_name].parent = nodes[parent_name]

root_nodes = [node for node in nodes.values() if node.is_root]

In [5]:
# Mostrar el árbol
for root_node in root_nodes:
    for pre, fill, node in RenderTree(root_node, style=AsciiStyle()):
        print(f"{pre}{node.name}")

root
|-- Activity
|   |-- Game
|   |   |-- BoardGame
|   |   +-- CardGame
|   |-- Sales
|   +-- Sport
|       |-- Athletics
|       |-- Boxing
|       |   |-- BoxingCategory
|       |   +-- BoxingStyle
|       +-- HorseRiding
|-- Agent
|   |-- Deity
|   |-- Employer
|   |-- Family
|   |   +-- NobleFamily
|   |-- Organisation
|   |   |-- Broadcaster
|   |   |   |-- BroadcastNetwork
|   |   |   |-- RadioStation
|   |   |   +-- TelevisionStation
|   |   |-- Company
|   |   |   |-- Airline
|   |   |   |-- Bank
|   |   |   |-- Brewery
|   |   |   |-- BusCompany
|   |   |   |-- Caterer
|   |   |   |-- LawFirm
|   |   |   |-- Publisher
|   |   |   |-- RecordLabel
|   |   |   +-- Winery
|   |   |-- EducationalInstitution
|   |   |   |-- College
|   |   |   |-- Library
|   |   |   |-- School
|   |   |   +-- University
|   |   |-- EmployersOrganisation
|   |   |-- GeopoliticalOrganisation
|   |   |-- GovernmentAgency
|   |   |-- Group
|   |   |   |-- Band
|   |   |   +-- ComedyGroup
|   |   |-- 

In [6]:
#Copiar árbol
root_nodes_copy = deepcopy(root_nodes)

In [7]:
# Definir el número máximo de niveles
def prune_tree(node, max_depth, current_depth=1):
    # Si estamos al nivel máximo, eliminar todos los hijos
    if current_depth >= max_depth:
        node.children = []
    else:
        # Recursivamente podar cada hijo
        for child in node.children:
            prune_tree(child, max_depth, current_depth + 1)

# Podar el árbol a X niveles
for root in root_nodes_copy:
    prune_tree(root, X)

# Mostrar el árbol podado
for pre, fill, node in RenderTree(root_nodes_copy[0]):
    print("%s%s" % (pre, node.name))

root
├── Activity
├── Agent
├── Altitude
├── AnatomicalStructure
├── Area
├── Award
├── Biomolecule
├── Blazon
├── ChartsPlacements
├── ChemicalSubstance
├── Colour
├── Currency
├── Demographics
├── Depth
├── Device
├── Diploma
├── Disease
├── ElectionDiagram
├── EthnicGroup
├── Event
├── Flag
├── Food
├── GeneLocation
├── GrossDomesticProduct
├── GrossDomesticProductPerCapita
├── Holiday
├── HumanDevelopmentIndex
├── Language
├── List
├── MeanOfTransportation
├── Media
├── Medicine
├── Name
├── PenaltyShootOut
├── PersonFunction
├── Place
├── Polyhedron
├── Population
├── PublicService
├── RouteStop
├── Species
├── SportCompetitionResult
├── SportsSeason
├── Statistic
├── TimePeriod
├── TopicalConcept
├── UnitOfWork
├── Unknown
└── Work


In [8]:
df_new = pd.read_csv('FinalDataset/final_dataset.csv')

In [9]:
def find_leaves(node):
    if not node.children:
        return [node]
    leaves = []
    for child in node.children:
        leaves.extend(find_leaves(child))
    return leaves

# Crear una lista con los nombres de las hojas del árbol podado
leaves_names = []
for root in root_nodes_copy:
    leaves = find_leaves(root)
    leaves_names.extend([leaf.name for leaf in leaves])

print(leaves_names)

['Activity', 'Agent', 'Altitude', 'AnatomicalStructure', 'Area', 'Award', 'Biomolecule', 'Blazon', 'ChartsPlacements', 'ChemicalSubstance', 'Colour', 'Currency', 'Demographics', 'Depth', 'Device', 'Diploma', 'Disease', 'ElectionDiagram', 'EthnicGroup', 'Event', 'Flag', 'Food', 'GeneLocation', 'GrossDomesticProduct', 'GrossDomesticProductPerCapita', 'Holiday', 'HumanDevelopmentIndex', 'Language', 'List', 'MeanOfTransportation', 'Media', 'Medicine', 'Name', 'PenaltyShootOut', 'PersonFunction', 'Place', 'Polyhedron', 'Population', 'PublicService', 'RouteStop', 'Species', 'SportCompetitionResult', 'SportsSeason', 'Statistic', 'TimePeriod', 'TopicalConcept', 'UnitOfWork', 'Unknown', 'Work']


In [10]:
# Filtrar las filas del DataFrame que pertenezcan a las hojas del árbol podado
filtered_df = df_new[df_new['Subclass'].isin(leaves_names)]

filtered_df = filtered_df.rename(columns={'Class': 'NewClass'})
num_labels = filtered_df['NewClass'].nunique()

print(num_labels)

# Guardar el DataFrame filtrado en un nuevo CSV
filtered_df.to_csv('FinalDataset/polished_dataset_realsub_'+str(num_labels)+'.csv', index=False)

33
